In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/lydiareader/git/dcds-500-project-2


'/Users/lydiareader/git/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [20]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_no_gaming = pd.Series(group_fairness(df), name='Lottery')
lottery_no_gaming

Consumer utility                  0.416673
Seller utility                    0.148627
average_num_got_shoes           394.000000
average_percent_got_shoes         0.043778
average_shoes_bought            596.000000
average_shoes_per_person          0.066222
abnormal_num_got_shoes           33.000000
abnormal_percent_got_shoes        0.033000
abnormal_shoes_bought            48.000000
abnormal_shoes_per_person         0.048000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            51.000000
wealthy_percent_got_shoes         0.051000
wealthy_shoes_bought            102.000000
wealthy_shoes_per_person          0.102000
influencer_num_got_shoes         27.000000
influencer_percent_got_shoes      0.054000
influencer_shoes_bought          27.000000
influencer_shoes_per_person       0.054000
reseller_num_got_shoes          106.000000
reseller_pe

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [21]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
first_no_gaming = pd.Series(group_fairness(df), name='First-come')
first_no_gaming

Consumer utility                  0.910674
Seller utility                    0.055210
average_num_got_shoes           509.000000
average_percent_got_shoes         0.056556
average_shoes_bought            787.000000
average_shoes_per_person          0.087444
abnormal_num_got_shoes           47.000000
abnormal_percent_got_shoes        0.047000
abnormal_shoes_bought            64.000000
abnormal_shoes_per_person         0.064000
special_num_got_shoes            16.000000
special_percent_got_shoes         0.064000
special_shoes_bought             32.000000
special_shoes_per_person          0.128000
wealthy_num_got_shoes            46.000000
wealthy_percent_got_shoes         0.046000
wealthy_shoes_bought             92.000000
wealthy_shoes_per_person          0.092000
influencer_num_got_shoes         25.000000
influencer_percent_got_shoes      0.050000
influencer_shoes_bought          25.000000
influencer_shoes_per_person       0.050000
reseller_num_got_shoes            0.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [22]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
invite_no_gaming = pd.Series(group_fairness(df), name='Invitation')
invite_no_gaming

Consumer utility                  0.458424
Seller utility                    0.363243
average_num_got_shoes           197.000000
average_percent_got_shoes         0.021889
average_shoes_bought            290.000000
average_shoes_per_person          0.032222
abnormal_num_got_shoes           18.000000
abnormal_percent_got_shoes        0.018000
abnormal_shoes_bought            25.000000
abnormal_shoes_per_person         0.025000
special_num_got_shoes             6.000000
special_percent_got_shoes         0.024000
special_shoes_bought             12.000000
special_shoes_per_person          0.048000
wealthy_num_got_shoes            29.000000
wealthy_percent_got_shoes         0.029000
wealthy_shoes_bought             58.000000
wealthy_shoes_per_person          0.058000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes           58.000000
reseller_pe

In [23]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [24]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_with_gaming = pd.Series(group_fairness(df), name='Lottery with gaming')
lottery_with_gaming

Consumer utility                  0.174680
Seller utility                    0.347610
average_num_got_shoes           133.000000
average_percent_got_shoes         0.014778
average_shoes_bought            225.000000
average_shoes_per_person          0.025000
abnormal_num_got_shoes           15.000000
abnormal_percent_got_shoes        0.015000
abnormal_shoes_bought            26.000000
abnormal_shoes_per_person         0.026000
special_num_got_shoes             5.000000
special_percent_got_shoes         0.020000
special_shoes_bought             10.000000
special_shoes_per_person          0.040000
wealthy_num_got_shoes            20.000000
wealthy_percent_got_shoes         0.020000
wealthy_shoes_bought             40.000000
wealthy_shoes_per_person          0.040000
influencer_num_got_shoes          6.000000
influencer_percent_got_shoes      0.012000
influencer_shoes_bought           6.000000
influencer_shoes_per_person       0.012000
reseller_num_got_shoes          323.000000
reseller_pe

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [25]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_consumer_df()
first_with_gaming = pd.Series(group_fairness(df), name='First-come with gaming')
first_with_gaming

Consumer utility                  0.825946
Seller utility                    0.170029
average_num_got_shoes           200.000000
average_percent_got_shoes         0.022222
average_shoes_bought            312.000000
average_shoes_per_person          0.034667
abnormal_num_got_shoes           16.000000
abnormal_percent_got_shoes        0.016000
abnormal_shoes_bought            23.000000
abnormal_shoes_per_person         0.023000
special_num_got_shoes            13.000000
special_percent_got_shoes         0.052000
special_shoes_bought             26.000000
special_shoes_per_person          0.104000
wealthy_num_got_shoes            53.000000
wealthy_percent_got_shoes         0.053000
wealthy_shoes_bought            106.000000
wealthy_shoes_per_person          0.106000
influencer_num_got_shoes         13.000000
influencer_percent_got_shoes      0.026000
influencer_shoes_bought          13.000000
influencer_shoes_per_person       0.026000
reseller_num_got_shoes           52.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [26]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
invite_with_gaming = pd.Series(group_fairness(df), name='Invitation with gaming')
invite_with_gaming

Consumer utility                  0.359331
Seller utility                    0.477852
average_num_got_shoes            30.000000
average_percent_got_shoes         0.003333
average_shoes_bought             60.000000
average_shoes_per_person          0.006667
abnormal_num_got_shoes            3.000000
abnormal_percent_got_shoes        0.003000
abnormal_shoes_bought             6.000000
abnormal_shoes_per_person         0.006000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes             8.000000
wealthy_percent_got_shoes         0.008000
wealthy_shoes_bought             16.000000
wealthy_shoes_per_person          0.016000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes          204.000000
reseller_pe

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [27]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='Lottery Cap1 with gaming')
lottery_cap1_with_gaming

Consumer utility                  0.170947
Seller utility                    0.352974
average_num_got_shoes           234.000000
average_percent_got_shoes         0.026000
average_shoes_bought            236.000000
average_shoes_per_person          0.026222
abnormal_num_got_shoes           33.000000
abnormal_percent_got_shoes        0.033000
abnormal_shoes_bought            33.000000
abnormal_shoes_per_person         0.033000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            47.000000
wealthy_percent_got_shoes         0.047000
wealthy_shoes_bought             47.000000
wealthy_shoes_per_person          0.047000
influencer_num_got_shoes         10.000000
influencer_percent_got_shoes      0.020000
influencer_shoes_bought          10.000000
influencer_shoes_per_person       0.020000
reseller_num_got_shoes          584.000000
reseller_pe

## Authentication

With auth, it's harder for reseller get fake acccounts

In [28]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='Lottery Authentication with gaming')
lottery_authen_with_gaming

Consumer utility                  0.208162
Seller utility                    0.321892
average_num_got_shoes           172.000000
average_percent_got_shoes         0.019111
average_shoes_bought            254.000000
average_shoes_per_person          0.028222
abnormal_num_got_shoes           18.000000
abnormal_percent_got_shoes        0.018000
abnormal_shoes_bought            27.000000
abnormal_shoes_per_person         0.027000
special_num_got_shoes             2.000000
special_percent_got_shoes         0.008000
special_shoes_bought              4.000000
special_shoes_per_person          0.016000
wealthy_num_got_shoes            20.000000
wealthy_percent_got_shoes         0.020000
wealthy_shoes_bought             40.000000
wealthy_shoes_per_person          0.040000
influencer_num_got_shoes          8.000000
influencer_percent_got_shoes      0.016000
influencer_shoes_bought           8.000000
influencer_shoes_per_person       0.016000
reseller_num_got_shoes          319.000000
reseller_pe

In [30]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='Lottery Cap1 and Auth with gaming')
lottery_authen_cap1_with_gaming

Consumer utility                  0.193318
Seller utility                    0.329925
average_num_got_shoes           282.000000
average_percent_got_shoes         0.031333
average_shoes_bought            282.000000
average_shoes_per_person          0.031333
abnormal_num_got_shoes           34.000000
abnormal_percent_got_shoes        0.034000
abnormal_shoes_bought            34.000000
abnormal_shoes_per_person         0.034000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            26.000000
wealthy_percent_got_shoes         0.026000
wealthy_shoes_bought             26.000000
wealthy_shoes_per_person          0.026000
influencer_num_got_shoes         14.000000
influencer_percent_got_shoes      0.028000
influencer_shoes_bought          14.000000
influencer_shoes_per_person       0.028000
reseller_num_got_shoes          573.000000
reseller_pe

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [32]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='Combined Distribution with gaming')
combined_distribution_with_gaming

Consumer utility                  0.548729
Seller utility                    0.373504
average_num_got_shoes           123.000000
average_percent_got_shoes         0.013667
average_shoes_bought            187.000000
average_shoes_per_person          0.020778
abnormal_num_got_shoes           17.000000
abnormal_percent_got_shoes        0.017000
abnormal_shoes_bought            24.000000
abnormal_shoes_per_person         0.024000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            32.000000
wealthy_percent_got_shoes         0.032000
wealthy_shoes_bought             64.000000
wealthy_shoes_per_person          0.064000
influencer_num_got_shoes        309.000000
influencer_percent_got_shoes      0.618000
influencer_shoes_bought         309.000000
influencer_shoes_per_person       0.618000
reseller_num_got_shoes           96.000000
reseller_pe

## Get Final Results

In [33]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [34]:
# uncomment to save results
# final_result.to_csv('project2_results.txt',sep = '\t') 
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
Lottery,0.416673,0.148627,394.0,0.043778,596.0,0.066222,33.0,0.033,48.0,0.048,...,102.0,0.102,27.0,0.054,27.0,0.054,106.0,0.0424,211.0,0.0844
First-come,0.910674,0.055210,509.0,0.056556,787.0,0.087444,47.0,0.047,64.0,0.064,...,92.0,0.092,25.0,0.050,25.0,0.050,0.0,0.0000,0.0,0.0000
Invitation,0.458424,0.363243,197.0,0.021889,290.0,0.032222,18.0,0.018,25.0,0.025,...,58.0,0.058,500.0,1.000,500.0,1.000,58.0,0.0232,115.0,0.0460
Lottery with gaming,0.174680,0.347610,133.0,0.014778,225.0,0.025000,15.0,0.015,26.0,0.026,...,40.0,0.040,6.0,0.012,6.0,0.012,323.0,0.1292,693.0,0.2772
First-come with gaming,0.825946,0.170029,200.0,0.022222,312.0,0.034667,16.0,0.016,23.0,0.023,...,106.0,0.106,13.0,0.026,13.0,0.026,52.0,0.0208,520.0,0.2080
Invitation with gaming,0.359331,0.477852,30.0,0.003333,60.0,0.006667,3.0,0.003,6.0,0.006,...,16.0,0.016,500.0,1.000,500.0,1.000,204.0,0.0816,418.0,0.1672
Lottery Cap1 with gaming,0.170947,0.352974,234.0,0.026000,236.0,0.026222,33.0,0.033,33.0,0.033,...,47.0,0.047,10.0,0.020,10.0,0.020,584.0,0.2336,669.0,0.2676
Lottery Authentication with gaming,0.208162,0.321892,172.0,0.019111,254.0,0.028222,18.0,0.018,27.0,0.027,...,40.0,0.040,8.0,0.016,8.0,0.016,319.0,0.1276,667.0,0.2668
Lottery Cap1 and Auth with gaming,0.193318,0.329925,282.0,0.031333,282.0,0.031333,34.0,0.034,34.0,0.034,...,26.0,0.026,14.0,0.028,14.0,0.028,573.0,0.2292,640.0,0.2560
Combined Distribution with gaming,0.548729,0.373504,123.0,0.013667,187.0,0.020778,17.0,0.017,24.0,0.024,...,64.0,0.064,309.0,0.618,309.0,0.618,96.0,0.0384,400.0,0.1600


In [35]:
final_result.to_csv('project-2-results-updated.csv')